In [ ]:
import pyorc
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
adcp = pd.read_csv('LR_WaterLab_AA_20240604_132107_edited_pp.csv', sep='\t')
# adcp = adcp.rename(columns={'Mean Speed (m/s)': 'v_len'})
print(adcp.columns)
adcp

In [ ]:
ds = xr.open_dataset('June/results/video_capture_2024-06-04_13-06-48_velocimetry_results.nc')
ds = ds.mean(dim="time", keep_attrs=True)

df = ds.to_dataframe().reset_index(); ds.close()
df['v_len'] = np.sqrt(df.v_x**2 + df.v_y**2)
df


In [ ]:
ds = xr.open_dataset('June/results/video_capture_2024-06-04_13-06-48_velocimetry_results.nc')

ds.velocimetry.mask.corr(inplace=True)
ds.velocimetry.mask.minmax(inplace=True)
ds.velocimetry.mask.rolling(inplace=True)
ds.velocimetry.mask.outliers(inplace=True)
ds.velocimetry.mask.variance(inplace=True)
ds.velocimetry.mask.angle(angle_tolerance=0.5*np.pi)
ds.velocimetry.mask.count(inplace=True)
ds.velocimetry.mask.window_mean(wdw=2, inplace=True, tolerance=0.5, reduce_time=True)

ds = ds.mean(dim="time", keep_attrs=True)

df_post = ds.to_dataframe().reset_index(); ds.close()
df_post['v_len'] = np.sqrt(df_post.v_x**2 + df_post.v_y**2)
df_post

In [ ]:
print('MEAN VELOCITY')
print(f'{'ADCP':<24}: {np.mean(adcp['v_len'])}')
print(f'{'PyORC':<24}: {np.mean(df['v_len'])}')
print(f'{'PyORC w/ post-processing':<24}: {np.mean(df_post['v_len'])}')

range = [0,2.5]
n=64

plt.figure(figsize=(7, 5))

sns.histplot(
    data=adcp,
    x='v_len',
    binrange=range,
    bins=n,
    label='ADCP'
)
sns.histplot(
    data=df,
    x='v_len',
    binrange=range,
    bins=n,
    label='PyORC'
)
sns.histplot(
    data=df_post,
    x='v_len',
    binrange=range,
    bins=n,
    label='PyORC w/ post-processing'
)

plt.xlabel('velocity')
plt.legend()